In [ ]:
import pandas as pd

# Asegúrate de usar el separador correcto (;)
df = pd.read_csv("20250212_Rendimiento_2024_20250131_WEB.csv", sep=";", low_memory=False)

print(df.shape)
df.head()


(3568930, 38)


,AGNO,RBD,DGV_RBD,NOM_RBD,COD_REG_RBD,NOM_REG_RBD_A,COD_PRO_RBD,COD_COM_RBD,NOM_COM_RBD,COD_DEPROV_RBD,...,COD_COM_ALU,NOM_COM_ALU,COD_RAMA,COD_SEC,COD_ESPE,PROM_GRAL,ASISTENCIA,SIT_FIN,SIT_FIN_R,COD_MEN
0,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"6,2",99,P,P,0
1,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,T,0
2,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"5,2",85,R,R,0
3,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"5,8",98,P,P,0
4,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,T,0


In [2]:
# Reemplaza ',' por '.' en columnas relevantes
df["PROM_GRAL"] = df["PROM_GRAL"].astype(str).str.replace(",", ".").astype(float)
df["ASISTENCIA"] = df["ASISTENCIA"].astype(str).str.replace(",", ".").astype(float)


In [4]:
# Convertir FEC_NAC_ALU a fecha
df["FEC_NAC_ALU"] = pd.to_datetime(df["FEC_NAC_ALU"], errors="coerce", format="%Y%m")

# Verifica fechas mal formateadas
df[df["FEC_NAC_ALU"].isnull()]


,AGNO,RBD,DGV_RBD,NOM_RBD,COD_REG_RBD,NOM_REG_RBD_A,COD_PRO_RBD,COD_COM_RBD,NOM_COM_RBD,COD_DEPROV_RBD,...,COD_COM_ALU,NOM_COM_ALU,COD_RAMA,COD_SEC,COD_ESPE,PROM_GRAL,ASISTENCIA,SIT_FIN,SIT_FIN_R,COD_MEN
3320626,2024,26062,2,COLEGIO DE AD. SAN JAVIER DE SAN MIGUEL,13,RM,131,13130,SAN MIGUEL,131,...,13129,SAN JOAQUÍN,0,0,0,0.0,0.0,Y,Y,0


In [5]:
# Convertir EDAD_ALU a número (forzando errores como NaN si hay valores no válidos)
df["EDAD_ALU"] = pd.to_numeric(df["EDAD_ALU"], errors="coerce")


In [6]:
# Eliminar filas con edad nula o negativa, o sin notas válidas
df = df[df["EDAD_ALU"] > 5]
df = df[df["PROM_GRAL"].notna()]
df = df[df["SIT_FIN"].isin(["P", "R", "Y"])]  # Aprobado, Reprobado, Retirado


In [7]:
# Filtrar datos válidos
df = df[df["ASISTENCIA"] > 0]
df = df[df["PROM_GRAL"] > 0]

In [8]:
# Eliminar filas con valores nulos en columnas clave
columnas_clave = ["MRUN", "GEN_ALU", "EDAD_ALU", "PROM_GRAL", "ASISTENCIA", "SIT_FIN"]
df = df.dropna(subset=columnas_clave)

In [12]:
# Crear dimensiones
df_dim_alumno = (
    df[["MRUN", "GEN_ALU", "FEC_NAC_ALU", "EDAD_ALU"]]
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
    .rename(columns={"index": "ID_ALUMNO"})
)

df_dim_establecimiento = (
    df[["RBD", "NOM_RBD", "COD_DEPE", "NOM_REG_RBD_A", "NOM_COM_RBD"]]
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
    .rename(columns={
        "index": "ID_ESTABLECIMIENTO",
        "NOM_RBD": "NOMBRE_ESTABLECIMIENTO",
        "COD_DEPE": "DEP_ADMIN",
        "NOM_REG_RBD_A": "REGION",
        "NOM_COM_RBD": "COMUNA"
    })
)

df_dim_tiempo = (
    df[["AGNO"]]
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
    .rename(columns={"index": "ID_TIEMPO"})
)
df_dim_tiempo["SEMESTRE"] = 1  # Ajusta si tienes semestre real

df_dim_estado_final = (
    df[["SIT_FIN", "SIT_FIN_R"]]
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
    .rename(columns={"index": "ID_ESTADO_FINAL"})
)

# Merge para hechos
df_hechos = (
    df
    # Alumno
    .merge(df_dim_alumno, on=["MRUN", "GEN_ALU", "FEC_NAC_ALU", "EDAD_ALU"])
    # Establecimiento
    .merge(df_dim_establecimiento, left_on=["RBD", "NOM_RBD", "COD_DEPE", "NOM_REG_RBD_A", "NOM_COM_RBD"],
           right_on=["RBD", "NOMBRE_ESTABLECIMIENTO", "DEP_ADMIN", "REGION", "COMUNA"])
    # Tiempo
    .merge(df_dim_tiempo, on="AGNO")
    # Estado final
    .merge(df_dim_estado_final, on=["SIT_FIN", "SIT_FIN_R"])
)

# Selecciona y renombra columnas finales
df_hechos = df_hechos[[
    "ID_ALUMNO",
    "ID_ESTABLECIMIENTO",
    "ID_TIEMPO",
    "ID_ESTADO_FINAL",
    "LET_CUR",      # Asumiendo que "CURSO" es esto
    "ASISTENCIA",
    "PROM_GRAL"
]].rename(columns={
    "LET_CUR": "CURSO",
    "PROM_GRAL": "PROMEDIO"
})

In [ ]:
from sqlalchemy import create_engine

# Crea la conexión a MySQL
usuario = "root"
contraseña = "trolldesu2"
host = "localhost"
puerto = "3306"
base_datos = "mineduc"

# Usa pymysql o mysqlclient según lo que instalaste
engine = create_engine(f"mysql+pymysql://{usuario}:{contraseña}@{host}:{puerto}/{base_datos}")


In [17]:
# Subir dimensiones
df_dim_alumno.to_sql("dim_alumno", engine, if_exists="replace", index=False)
df_dim_establecimiento.to_sql("dim_establecimiento", engine, if_exists="replace", index=False)
df_dim_tiempo.to_sql("dim_tiempo", engine, if_exists="replace", index=False)
df_dim_estado_final.to_sql("dim_estado_final", engine, if_exists="replace", index=False)

# Subir hechos
df_hechos.to_sql("hechos_rendimiento", engine, if_exists="replace", index=False)


3045217